# STEP1_Scraper_RDS

## STEP1_0_SetUp

In [1]:
# Standard library imports
import json
import re
import time
import random
from collections import deque
from datetime import datetime
from urllib.parse import urljoin, urlparse

# Third-party library imports
import boto3
import pandas as pd
import requests
from bs4 import BeautifulSoup
import mysql.connector
import sqlite3

# Local module imports from 'scripts' directory
from scripts.create_db import create_tables
from scripts.database_scripts import insert_job_postings, insert_data
from scripts.fetch import JobSearchRetriever, JobDetailRetriever
from scripts.helpers import clean_job_postings

loading full_matcher ...
loading abv_matcher ...
loading full_uni_matcher ...
loading low_form_matcher ...
loading token_matcher ...


## STEP1_1_Create_RDS (Skip this step if RDS has been created)

In [2]:
# Initial Setup
rds_client = boto3.client('rds')
iam_client = boto3.client('iam')
role = iam_client.get_role(RoleName='LabRole')

USERNAME = 'username'
PASSWORD = 'password'

# Create RDS instance
try:
    response = rds_client.create_db_instance(
        DBInstanceIdentifier='relational-db',
        DBName='linkedin',
        MasterUsername=USERNAME,
        MasterUserPassword=PASSWORD,
        DBInstanceClass='db.t3.medium',
        Engine='MySQL',
        AllocatedStorage=5
    )
    print("RDS instance created successfully:", response)
    
    # Wait until DB is available to continue
    rds_client.get_waiter('db_instance_available').wait(DBInstanceIdentifier='relational-db')
    
    # Describe where DB is available and on what port
    db = rds_client.describe_db_instances()['DBInstances'][0]
    ENDPOINT = db['Endpoint']['Address']
    PORT = db['Endpoint']['Port']
    DBID = db['DBInstanceIdentifier']
    
    print(DBID,
        "is available at", ENDPOINT,
        "on Port", PORT,
        )  
     
except Exception as e:
    print("Error creating RDS instance:", e)

Error creating RDS instance: An error occurred (DBInstanceAlreadyExists) when calling the CreateDBInstance operation: DB instance already exists


# Step1_2_Scraping_Job_Overview

In [3]:
# Connect with RDS
USERNAME = 'username'
PASSWORD = 'password'
ENDPOINT = 'relational-db.cfxyluhsb7bh.us-east-1.rds.amazonaws.com'
PORT = 3306

conn =  mysql.connector.connect(host=ENDPOINT, user=USERNAME, passwd=PASSWORD, port=PORT, database='linkedin')
cursor = conn.cursor()

# Create database tables
create_tables(conn, cursor)

# Set parameters for job overview scraper
job_searcher = JobSearchRetriever()
sleep_times = deque(maxlen=5)
first = True
sleep_factor = 3

# Scrape job overview
while True:
    # Get all results from the scraper and check the duplicated results
    all_results = job_searcher.get_jobs()
    query = "SELECT job_id FROM jobs WHERE job_id IN ({})".format(','.join(['%s'] * len(all_results)))
    cursor.execute(query, list(all_results.keys()))
    result = cursor.fetchall()
    result = [r[0] for r in result]
    new_results = {job_id: job_info for job_id, job_info in all_results.items() if job_id not in result}
    
    # Insert new results into database
    insert_job_postings(new_results, conn, cursor)
    total_non_sponsored = len([x for x in all_results.values() if x['sponsored'] is False])
    new_non_sponsored = len([x for x in new_results.values() if x['sponsored'] is False])
    print('{}/{} NEW RESULTS | {}/{} NEW NON-PROMOTED RESULTS'.format(
        len(new_results), len(all_results), new_non_sponsored, total_non_sponsored))
    
    # Adjust scrape sleep time based on the number of new results
    if not first:
        seconds_per_job = sleep_factor/max(len(new_results), 1)
        sleep_factor = min(seconds_per_job * total_non_sponsored * .75, 200)
    first = False

    # Sleep to try to avoid blockage
    print('Sleeping For {} Seconds...'.format(min(200, sleep_factor)))
    time.sleep(min(200, sleep_factor))
    print('Resuming...')

# It will keep scraping until being interrupted

Jobs insertion/update completed.
100/100 NEW RESULTS | 100/100 NEW NON-PROMOTED RESULTS
Sleeping For 3 Seconds...
Resuming...
Jobs insertion/update completed.
53/100 NEW RESULTS | 53/100 NEW NON-PROMOTED RESULTS
Sleeping For 4.245283018867925 Seconds...
Resuming...
Jobs insertion/update completed.
1/100 NEW RESULTS | 1/100 NEW NON-PROMOTED RESULTS
Sleeping For 200 Seconds...


KeyboardInterrupt: 

## STEP1_3_Scrape_Job_Datail

In [5]:
# Connect with RDS
USERNAME = 'username'
PASSWORD = 'password'
ENDPOINT = 'relational-db.cfxyluhsb7bh.us-east-1.rds.amazonaws.com'
PORT = 3306
conn =  mysql.connector.connect(host=ENDPOINT, user="username", passwd="password", port=PORT, database='linkedin')
cursor = conn.cursor()

# Set parameters for job detail scraper
job_detail_retriever = JobDetailRetriever()
SLEEP_TIME = 60
MAX_UPDATES = 25

# Scrape job detail
while True:
    # Get job postings hasn't been scraped before
    query = "SELECT job_id FROM jobs WHERE scraped = 0"
    cursor.execute(query)
    result = cursor.fetchall()
        
    if not result:
        print("No more job to process.")
        break

    result = [r[0] for r in result]

    # Get & clean job detail and insert it into database
    details = job_detail_retriever.get_job_details(random.sample(result, min(MAX_UPDATES, len(result))))
    details = clean_job_postings(details)
    insert_data(details, conn, cursor)
    print('UPDATED {} VALUES IN DB'.format(len(details)))

    # Sleep to try to avoid blockage
    print('Sleeping For {} Seconds...'.format(SLEEP_TIME))
    time.sleep(SLEEP_TIME)
    print('Resuming...')

# Close MySQL connection if there's no more job to process
if conn.is_connected():
    cursor.close()
    conn.close()
    print("MySQL connection is closed")

Job 3934281333 done
Job 3928874752 done
Job 3934278890 done
Job 3934279657 done
Job 3928874758 done
Job 3934963664 done
Job 3928870071 done
Job 3934966393 done
Job 3928859973 done
Job 3928867514 done
Job 3934276562 done
Job 3934961859 done
Job 3928883041 done
Job 3934966182 done
Job 3934280486 done
Job 3928875696 done
Job 3928866577 done
Job 3934283035 done
Job 3934966226 done
Job 3934279674 done
Job 3934965589 done
Job 3934278778 done
Job 3928877659 done
Job 3934961796 done
Job 3928869149 done
loading full_matcher ...
loading abv_matcher ...
loading full_uni_matcher ...
loading low_form_matcher ...
loading token_matcher ...


/Users/qxlin/Library/CloudStorage/GoogleDrive-qxlin@uchicago.edu/My Drive/UChi/MACS-30123/.venv/lib/python3.10/site-packages/skillNer/utils.py:99: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  vec_similarity = token1.similarity(token2)


loading full_matcher ...
loading abv_matcher ...
loading full_uni_matcher ...
loading low_form_matcher ...
loading token_matcher ...


KeyboardInterrupt: 

In [9]:
# Check if the scraper get info successfully - check the ner_skills table as an example

try:
    conn =  mysql.connector.connect(host=ENDPOINT, user=USERNAME, passwd=PASSWORD, port=PORT, database='linkedin')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM ner_skills")

        # Print 5 records from the table 
        print("5 rows from the table ner_skills:")
        rows = cursor.fetchmany(size=5)
        for row in rows:
            print(row)

except Exception as e:
    print("Error while connecting to MySQL", e)

5 rows from the table ner_skills:
(3928861779, 'customer service, organizational strategy, microsoft outlook, primary care, internet service, business requirement, primary care, primary care, preventive care, integrate care, behavioral health, primary care, leadership development, CRM')
(3928865930, 'retail sale, customer service, retail sale, retail sale, customer experience, retail sale, cash register, point of sale, reach')
(3928867512, 'greenhouse gas, policy analysis, sustainability strategy, sustainability planning, public policy, carbon accounting, project management, greenhouse gas, write communication, microsoft office, time management, organizational skill, business ethic, service provider, religious belief, San, San, com, Act')
(3928871251, 'customer service, retail sale, customer experience, retail sale, cash register, point of sale, reach')
(3928877269, 'market share, safety culture, compliance training, case management, bloodborne pathogen, asbestos abatement, medical sur